In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `C:\Users\nsthtz\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...


In [47]:
using DelimitedFiles
#data = readdlm(joinpath("dummyMajor.csv"),';')
#data = readdlm(joinpath("dummyMajorA#.csv"),';')
data = readdlm(joinpath("dummyMajorMultiple.csv"),';')
#data = readdlm(joinpath("dummyMinor.csv"),',')
#data = readdlm(joinpath("dummyMinorA#.csv"),';')
data = round.(Int, data)

128×24 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮         

# # Convert piano roll into timestepx12 tonal mapping


In [48]:
#

tonal_map = zeros(Int8, size(data, 2), 12)
i=1
data[:, 1]

for i in 1:size(data, 2)
    active_notes = findall(x->x==1, data[:,i])
    #println(active_notes)
    for note in active_notes
        tonal_map[i, note%12+1] += 1
    end
    #println(timestep_chords[i,:])
end

tonal_map

24×12 Array{Int8,2}:
 1  1  1  0  1  0  0  0  0  0  0  0
 0  0  1  0  0  1  0  0  1  1  0  0
 1  0  0  0  2  0  0  0  1  0  0  0
 0  0  0  0  0  2  0  0  0  1  1  0
 1  0  0  0  0  1  0  1  0  0  0  0
 1  0  0  1  0  0  0  0  0  2  0  0
 0  0  0  1  0  0  0  0  0  0  0  1
 1  0  0  0  0  1  0  0  0  0  0  0
 1  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  1  0  0  0  1  0  0  0  1
 1  0  1  0  0  0  0  0  0  1  0  0
 1  0  0  0  0  0  0  1  0  0  1  0
 0  0  0  0  0  2  0  0  0  0  0  0
 0  0  1  0  1  0  0  0  0  1  0  0
 0  0  1  0  0  0  1  0  1  0  0  0
 2  0  1  0  0  0  0  0  2  0  0  0
 0  0  1  0  0  1  0  0  0  0  0  1
 1  0  0  1  0  0  0  0  0  0  0  0
 0  1  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  1
 1  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  1  0  0
 0  0  0  0  1  0  0  0  0  0  0  1
 1  1  0  0  0  0  0  0  0  0  0  0

# # Pattern matcher to identify the best-fit key and scale from existing notes

In [49]:
# Functions and Global

pattern = [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]

# Weight of core notes (1, 4, 5) doubled
majorpattern = [1, 0, 0.5, 0, 0.5, 1, 0, 1, 0, 0.5, 0, 0.5]
minorpattern = [0.5, 0, 1, 0, 1, 0.5, 0, 0.5, 0, 1, 0, 0.5]

Keys = Dict{Integer, String}(1 => "C", 2 => "C#", 3 => "D", 4 => "D#", 5 => "E", 6 => "F", 7 => "F#", 8 => "G", 9 => "G#", 10 => "A", 11 => "A#", 12 => "B")


#Score In: Tx12 Matrix, pattern. Out: Best position(s)
function score_key(matrix, pattern)
    counter = zeros(Int8, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    println("Key Counter is ", counter)
    
    key_score = zeros(Int8, 12, 1)
    
    for i in 1:size(pattern, 1)
        for j in 1:size(pattern, 1)
            key_score[i] += circshift(pattern, i-1)[j]*counter[j]
        end
    end
    println("Key Score is ", key_score)
    return key_score
end

# Find maximum from list
function findmaximum(list)
    positions = []
    max_value = maximum(list)
    for (i, x) in enumerate(list)
        if x == max_value
            push!(positions, i)
        end
    end
    return positions
end

# Find an optimal scale
function score_tonality(matrix, positions)
    
    counter = zeros(Int8, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    
    tonality_scores = []
    
    for position in positions
        counter_shifted = circshift(counter, (0,-(position-1)))
        println("Position Counter for ", position, " is ", counter_shifted)
        
        major_score = 0
        minor_score = 0
        
        for i in 1:size(counter_shifted, 2)
            major_score += majorpattern[i]*counter_shifted[i]
            minor_score += minorpattern[i]*counter_shifted[i]
        end

        if major_score > minor_score
            push!(tonality_scores, [position, major_score, 1])
        else
            push!(tonality_scores, [position, minor_score, 0])
        end
        
    end 
    
    return tonality_scores
end

score_tonality (generic function with 1 method)

In [50]:
# Score matrix after pattern
key_score_result = score_key(tonal_map, pattern)
# score_result = [9, 5, 18, 9, 23, 18, 8, 2, 7, 16, 12, 11]

# Find positions of maximum score
positions = findmaximum(key_score_result)
sort!(positions)

# # Use positions to find the best key

tonality_results = score_tonality(tonal_map, positions)

# [Position (1 = C), Score, Scale (1 = Major, 0 = Minor)]
best_key = [0, 0, 1]

# In case of tied scores, check each for the best scale
for result_group in tonality_results
    if best_key[2] < result_group[2]
        best_key[1] = result_group[1]
        best_key[2] = result_group[2]
        best_key[3] = result_group[3]
    end
end
    
#Save Key
println(best_key)
if best_key[3] == 1
    key = best_key[1]
    key_print = Keys[best_key[1]]
    scale = "Major"
else
    key = (12+best_key[1]-3)%12
    key_print = Keys[key]
    scale = "Minor"
end
    
println(key, " ", key_print, " ", scale)

Key Counter is Int8[13 3 7 4 5 10 2 5 5 7 2 5]
Key Score is Int8[52; 39; 34; 46; 31; 49; 31; 44; 42; 34; 48; 26]
Position Counter for 1 is Int8[13 3 7 4 5 10 2 5 5 7 2 5]
[1, 40, 1]
1 C Major


# # Define patterns for chord functions relative to key

In [155]:
# Function entry: "Name" => ([notes], [Bass note], [Double/Triple notes], [Rhythmical Weight (Strong, Weak, Off)])
functions = Dict{String, Array{Int64, 1}}

if scale == "Major"
#Major  =  [1, 0, 2, 0, 3, 4, 0, 5, 0, 6, 0, 7]
#
    
    functions = Dict(
    "T"    =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O"]),
    "S"    =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "D"    =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "T-5"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O"]),
    "D-5"  =>  (circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "T/3"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((4+(key-1))%12)+1, 3) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["S", "W", "O"]),
    "S/3"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((9+(key-1))%12)+1, 3) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "D/3"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "T/5"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 5) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["W", "O"]),
    "S/5"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 5) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["W", "O"]),
    "D/5"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["W", "O"]),
    "D64"  =>  (circshift([4, 0, 0, 0, 6, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
    "T64"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 0, 0, 6, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
    "T54"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
    "D54"  =>  (circshift([4, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
    "D7"   =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
    "D7/3" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ nothing ], ["S", "W", "O"]),
    "D7/5" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ nothing ], ["W", "O"]),
    "D7/7" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((5+(key-1))%12)+1, 7) ], [ nothing ], ["S", "W", "O"]),
    "D7-1" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 0, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((5+(key-1))%12)+1, 7) ], ["S", "W", "O"]),
    "D7-5" =>  (circshift([0, 0, 0, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "S65"  =>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
    "S65/6"=>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 6) ], [ nothing ], ["S", "W", "O"]),
    "S6"   =>  (circshift([0, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "Ss"   =>  (circshift([0, 0, 1, 0, 0, 3, 0, 0, 0, 5, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 1) ], [ (((2+(key-1))%12)+1, 1), (((5+(key-1))%12)+1, 3) ], ["S", "W", "O"]),
    "Ts"   =>  (circshift([3, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0], (key-1)), [ (((9+(key-1))%12)+1, 1) ], [ (key, 3), (((9+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "Tm"   =>  (circshift([0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 5], (key-1)), [ (((4+(key-1))%12)+1, 1) ], [ (((4+(key-1))%12)+1, 1), (((7+(key-1))%12)+1, 3) ], ["S", "W", "O"]))

elseif scale == "Minor"
#                         [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11]
#Minor  =                 [1, 0, 2, 3, 0, 4, 0, 5, 6, 0, 7, 7+]              
    functions = Dict(
    "T"    =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O"]),
    "S"    =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "D"    =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 3, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "T-5"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O"]),
    "D-5"  =>  (circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "T/3"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((3+(key-1))%12)+1, 3) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["S", "W", "O"]),
    "S/3"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((8+(key-1))%12)+1, 3) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "D/3"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 3, 0], (key-1)), [ (((10+(key-1))%12)+1, 3) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "T/5"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 5) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["W", "O"]),
    "S/5"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 5) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["W", "O"]),
    "D/5"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 3, 0], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["W", "O"]),
    "D64"  =>  (circshift([4, 0, 0, 6, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
    "T64"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 0, 6, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
    "T54"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
    "D54"  =>  (circshift([4, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
    "D7"   =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 3, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
    "D7/3" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 3, 0], (key-1)), [ (((10+(key-1))%12)+1, 3) ], [ nothing ], ["S", "W", "O"]),
    "D7/5" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 3, 0], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ nothing ], ["W", "O"]),
    "D7/7" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 3, 0], (key-1)), [ (((5+(key-1))%12)+1, 7) ], [ nothing ], ["S", "W", "O"]),
    "D7-1" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 0, 0, 0, 3, 0], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((5+(key-1))%12)+1, 7) ], ["S", "W", "O"]),
    "D7-5" =>  (circshift([0, 0, 0, 0, 0, 7, 0, 1, 0, 0, 3, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "S65"  =>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
    "S65/6"=>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 6) ], [ nothing ], ["S"]),
    "S6"   =>  (circshift([0, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "Dm"   =>  (circshift([0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 1, 0], (key-1)), [ (((10+(key-1))%12)+1, 1) ], [ (((10+(key-1))%12)+1, 1), (((2+(key-1))%12)+1, 3) ], ["S", "W", "O"]),
    "Ts"   =>  (circshift([3, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0], (key-1)), [ (((8+(key-1))%12)+1, 1) ], [ (key, 3), (((8+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "Tm"   =>  (circshift([0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 5, 0], (key-1)), [ (((3+(key-1))%12)+1, 1) ], [ (((3+(key-1))%12)+1, 1), (((7+(key-1))%12)+1, 3) ], ["S", "W", "O"]))
       
end
      
for k in functions
    println(k[1], " ", k[2])
end

#      [C, C#, D, D#, E, F, F#, G, G#, A, A#, B]
#      [1, 2,  3, 4,  5, 6, 7,  8, 9, 10, 11, 12]
#      [2, 0,  3, 4,  0, 5, 0,  6, 0, 7,  1,  0]

S65/6 ([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], Tuple{Int64,Int64}[(3, 6)], Nothing[nothing], ["S", "W", "O"])
S/5 ([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], Tuple{Int64,Int64}[(1, 5)], Tuple{Int64,Int64}[(1, 5), (6, 1)], ["W", "O"])
T ([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], Tuple{Int64,Int64}[(1, 1)], Tuple{Int64,Int64}[(1, 1)], ["S", "W", "O"])
Ts ([3, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0], Tuple{Int64,Int64}[(10, 1)], Tuple{Int64,Int64}[(1, 3), (10, 1)], ["S", "W", "O"])
S/3 ([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], Tuple{Int64,Int64}[(10, 3)], Tuple{Int64,Int64}[(1, 5), (6, 1)], ["S", "W", "O"])
T/5 ([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], Tuple{Int64,Int64}[(8, 5)], Tuple{Int64,Int64}[(1, 1), (8, 5)], ["W", "O"])
Ss ([0, 0, 1, 0, 0, 3, 0, 0, 0, 5, 0, 0], Tuple{Int64,Int64}[(3, 1)], Tuple{Int64,Int64}[(3, 1), (6, 3)], ["S", "W", "O"])
D7/5 ([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], Tuple{Int64,Int64}[(3, 5)], Nothing[nothing], ["W", "O"])
T/3 ([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], Tuple{Int64,Int64}[(5,

# # Analyze tonal map and calculate hamming distances for each timestep

In [152]:
##### DISTANCE FUNCTIONS NOT USED

function hamming(input, target)
    counter = 0
    for i in 1:size(input, 1)
        if input[i] != target[i]
            counter = counter+1
        end
    end
    return counter
end

## What is the distance to transform an input into a function NOT USED
    
function distance(input, target)
    distance = 0
    target_padded = copy(target)
    target_padded = append!(append!(target_padded, target), target)[1:36]
    for i in 1:size(input, 1)
        if input[i] > 0
            if target[i] > 0
                for x in 0:2
                    target_padded[i+(12*x)] = 0
                end
            else
                for j in 1:6
                    if target_padded[12+i-j] > 0
                        distance += j
                        break
                    elseif target_padded[12+i+j] > 0
                        distance += j
                        break
                    end
                end
            end
        end
    end
    
    return distance
end

## What is the distance of finding a valid function within the input NOT USED
        
function distance(input, target)
    distance = 0
    input_padded = append!(append!(input, input), input)[1:36]
    for i in 1:size(target, 1)
        if target[i] > 0
            if input[i] == 0
                for j in 1:6
                    if input_padded[12+i-j] > 0
                        distance += j
                        break
                    elseif input_padded[12+i+j] > 0
                        distance += j
                        break
                    end
                end
            end
        end
    end
                
    return distance
end
        

##### TODO: Adjust distance function so that it counts cumulative distance from one note in target to the nearest in input, Left-Left matching, remove upon match                

function distance(input, target)
    input = copy(input)
    distance = 0
    input_padded = append!(append!(input, input), input)[1:36]
    for i in 1:size(target, 1)
        if target[i] > 0
            if input_padded[12+i] == 0
                for j in 1:6
                    if input_padded[12+i-j] > 0
                        distance += j
                        for k in 1:2
                            input_padded[k*12+i-j] -= 1
                        end 
                        break
                    elseif input_padded[12+i+j] > 0
                        distance += j
                        for k in 0:1
                            input_padded[k*12+i+j] -= 1
                        end
                        break
                    end
                end
            elseif input_padded[12+i] > 0
                for k in 0:2
                    input_padded[k*12+i] -= 1
                end
            end
        end
    end
                
    return distance
end                  

# 

chords = keys(functions)
function_potential = []

for i in 1:size(tonal_map, 1)         
    function_pool = []
    for chord in chords
        chord_distance = distance(tonal_map[i,:], functions[chord][1])
        push!(function_pool, (chord_distance, chord))
    end
    push!(function_potential, sort(function_pool))
end

println(function_potential[1])

## Naive function label
                    
chosen_chords = []                    
                  
    
for timestep in 1:size(function_potential, 1)
    append!(chosen_chords, [function_potential[timestep][1][2]])
end
                    
println(chosen_chords)


Any[(0, "T-5"), (2, "D7-1"), (3, "D54"), (4, "D"), (4, "D-5"), (4, "D/3"), (4, "D/5"), (4, "S6"), (4, "Ss"), (4, "Ts"), (5, "D64"), (5, "S"), (5, "S/3"), (5, "S/5"), (5, "S65"), (5, "S65/6"), (5, "T"), (5, "T/3"), (5, "T/5"), (5, "T64"), (6, "T54"), (6, "Tm"), (7, "D7-5"), (8, "D7"), (8, "D7/3"), (8, "D7/5"), (8, "D7/7")]
Any["T-5", "S6", "T-5", "S", "T54", "T-5", "D54", "S", "D-5", "D-5", "D54", "D-5", "D7-5", "S6", "D7", "D54", "D7-1", "D54", "D7", "D-5", "S", "D-5", "D64", "D54"]


# # Construct the search space for note distribution to voices

In [151]:
using Combinatorics

# Finds and returns all possible distributions of the valid note classes for a given function

function getVoicingPermutations(func)
    voicing_permutations = []
    voicings_initial = []
    
    function_map = functions[func][1]
    double_note = functions[func][3]
    bass_note = functions[func][2][1]

    active_notes = sort!([(i, x) for (i, x) in enumerate(function_map) if x > 0], by = x -> x[2])   # [position, value]
    
    filter!(e->e != bass_note, active_notes)
        
    voicing_sans_bass = copy(active_notes)
            
    if size(active_notes, 1) == 1
        push!(voicing_sans_bass, bass_note)
        push!(voicing_sans_bass, bass_note)
        push!(voicings_initial, voicing_sans_bass)
    elseif size(active_notes, 1) == 2
        for i in 1:size(double_note, 1)
            voicing_sans_bass = copy(active_notes)
            push!(voicing_sans_bass, double_note[i])
            push!(voicings_initial, voicing_sans_bass)
        end
    elseif size(active_notes, 1) == 3
        push!(voicings_initial, voicing_sans_bass)
    end
                    
    for entry in voicings_initial
        voicings = collect(permutations(entry))
        for j in voicings
            push!(voicing_permutations, j)
        end
    end
    
    voicing_permutations = unique(voicing_permutations)   
    for p in voicing_permutations
        pushfirst!(p, bass_note)
    end
    
    return voicing_permutations
end
        
# Generates the candidate solutions of note distribution for a given function
               
function generateCandidates(func)
    voicing_permutations = getVoicingPermutations(func)
    valid_voicings = []
    note_repository = []   # [[all C's], [all C#'s], [all D's], ..., [all B's]]
    for note_class in 1:12
        push!(note_repository, [x for x in 40:80 if x%12 == (note_class-1)])
    end
    #println(note_repository)
    
    
    for permutation in voicing_permutations
        #println(permutation)
        voicing = []
        for b in note_repository[permutation[1][1]]
            if b in 40:60
                for t in note_repository[permutation[2][1]]
                    if t in 47:68 && t >= b && t-b <= 20
                        for a in note_repository[permutation[3][1]]
                            if a in 52:71 && a >= t && a-t <= 12                                 
                                for s in note_repository[permutation[4][1]]
                                    if s in 59:80 && s >= a && s-a <= 12  
                                        voicing = []
                                        push!(voicing, (b, permutation[1][2]), (t, permutation[2][2]), (a, permutation[3][2]), (s, permutation[4][2])) 
                                        push!(valid_voicings, voicing)
                                    end                                
                                end           
                            end    
                        end
                        
                    end
                end       
            end
        end
    end
    
    ## Enforce Special rules
    
    if (func == "T-5") || (func == "D-5")
        placeholder = []
        for voicing in valid_voicings
            if (voicing[4][2] == 1) && (voicing[4][1] - voicing[3][1] == 8)
                push!(placeholder, voicing)
            end
        end
        valid_voicings = placeholder
    end
    
    return(valid_voicings)
end

function_permutations = Dict{String, Array{Any, 1}}()

for keys in keys(functions)
    function_permutations[keys] = unique(generateCandidates(keys))
end

display(function_permutations["D7-5"])

14-element Array{Any,1}:
 Any[(43, 1), (59, 3), (65, 7), (67, 1)]
 Any[(55, 1), (59, 3), (65, 7), (67, 1)]
 Any[(43, 1), (47, 3), (55, 1), (65, 7)]
 Any[(43, 1), (59, 3), (67, 1), (77, 7)]
 Any[(55, 1), (59, 3), (67, 1), (77, 7)]
 Any[(43, 1), (53, 7), (59, 3), (67, 1)]
 Any[(55, 1), (65, 7), (71, 3), (79, 1)]
 Any[(43, 1), (53, 7), (55, 1), (59, 3)]
 Any[(55, 1), (65, 7), (67, 1), (71, 3)]
 Any[(43, 1), (55, 1), (59, 3), (65, 7)]
 Any[(55, 1), (55, 1), (59, 3), (65, 7)]
 Any[(55, 1), (67, 1), (71, 3), (77, 7)]
 Any[(43, 1), (55, 1), (65, 7), (71, 3)]
 Any[(55, 1), (55, 1), (65, 7), (71, 3)]

# # Rhythm function

In [148]:
function rhythmify(time_signature, original) # (beats per bar, subdivisions) (3,1), (3, 2), (4,1), (4, 2)
    
    rhythm_map = []
    
    beat_remainder = size(original, 2) % (time_signature[1] * time_signature[2])
    
    rhythms = Dict(
        (3, 1) => ["S", "W", "W"],
        (3, 2) => ["S", "O", "W", "O", "W", "O"],
        (4, 1) => ["S", "W", "S", "W"],
        (4, 2) => ["S", "O", "W", "O", "S", "O",  "W", "O"]
    )

    ## Create basic rhythm map
    if beat_remainder == time_signature[2]
        for i in 1:time_signature[2]
            push!(rhythm_map, rhythms[time_signature][end - (time_signature[2] - i)])
        end
            
        for i in 0:(size(original, 2) - 1-time_signature[2])
            push!(rhythm_map, rhythms[time_signature][(i % (time_signature[1]*time_signature[2]) + 1)])
        end
    else
        for i in 0:(size(original, 2) - 1)
            push!(rhythm_map, rhythms[time_signature][(i % (time_signature[1]*time_signature[2]) + 1)])
        end
    end
    
    return rhythm_map
end

println(rhythmify((3, 2), data[:, 5:end]))

Any["W", "O", "S", "O", "W", "O", "W", "O", "S", "O", "W", "O", "W", "O", "S", "O", "W", "O", "W", "O"]


In [147]:
#timestep = [(function_cost, "function_name"), [(bass_note, bass_interval), (tenor_note, tenor_interval), (alto_note, alto_interval), (soprano_note, soprano_interval)]]

function voicing_cost(previous_timestep, current_timestep)
    
    current_notes = current_timestep[2]
    previous_notes = previous_timestep[2]
    
    cost = current_timestep[1][1]
    
    distance = [current_notes[1][1] - previous_notes[1][1], current_notes[2][1] - previous_notes[2][1], current_notes[3][1] - previous_notes[3][1], current_notes[4][1] - previous_notes[4][1]]

    # Avalanche check
    
    if abs(reduce(+, map(x -> sign(x), distance))) == 4
        cost += 10000
    end
    
    # voice movement cost
    
    cost += reduce(+, map(x -> abs(x) > 2 ? abs(x)-2 : 0 , distance))
    
    # distance from original cost
    
    # TODO: Might mess things up
    
    # Prevent chord repetition
    
    if reduce(+, map(x -> abs(x), distance)) == 0
        cost += 10000
    end
    
    return cost
    
end

function function_cost()
    
end

previous_timestep = [(2, "T"), [(43, 1), (59, 3), (65, 7), (67, 1)]]
current_timestep = [(2, "T"), [(43, 1), (59, 3), (65, 7), (67, 1)]]

cost(previous_timestep, current_timestep)

10002

# # DFS Branch and Bound ?

In [180]:
function prepare(data, functions, function_potential)
    
    Tonics = ["T", "T/3", "T/5", "T-5"]
    Tonics_dependant = ["T64", "T54"]
    Subdominants = ["S", "S/3", "S/5", "S65", "S65/6", "S6"]
    Dominants = ["D", "D/3", "D/5", "D-5", "D64", "D5", "D54", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7"]
    Mediants = ["Ts", "Tm", "Ss", "Dm"]
    
    start = ["T"]
    leading_end = ["D", "D7"]
    final = ["T"]
    
    # Local copy of the original data
    data_local = deepcopy(data)
    # Local copy of dictionary with global function rules : dict[key] = ([1: tonal_map], [2: bass note], [3: double/triple note], [4: Rhythmical rules])
    functions_local = deepcopy(functions)  
    
    # Local copy of list of candidate functions for each timestep [[timestep1], [timestep2], ... ]
    # Each index contains a sorted list of tuples which determine function costs for the given timestep : function_potential_local[i] = [(function_cost, "function_name"), ...]
    function_potential_local = deepcopy(function_potential[1:size(data_local, 2)])
    
    # Create rhythm map for piece
    time_signature = (4, 2)
    rhythm_map_local = rhythmify(time_signature, data)
    println(rhythm_map_local)
    
    # Locate and enforce starting and ending Tonic, and dominant in final cadence
    
    start_index = findfirst(x -> x == "S", rhythm_map_local)
    final_index = findlast(x -> x == "S", rhythm_map_local)
    leading_index = findlast(x -> x == "W" || x == "S", rhythm_map_local[1:final_index-1])
    
    filter!(x -> x[2] in start, function_potential_local[start_index])
    for i in leading_index:final_index-1
        filter!(x -> x[2] in Dominants, function_potential_local[i])
    end
    filter!(x -> x[2] in leading_end, function_potential_local[final_index-1])
    for i in final_index:size(data, 2)
        filter!(x -> x[2] in final, function_potential_local[i])
    end
    
    # Prune function_potential by beat restrictions for each chord
    
    for (i, potential) in enumerate(function_potential_local)
        for (j, (cost, chord)) in enumerate(potential)
            if !(rhythm_map_local[i] in functions_local[chord][4])
                filter!(x -> x[2] != chord, potential)
            end
        end
    end
    
    #display(function_potential_local)
    
    best_solution = []
    best_solution = BB(function_potential_local, rhythm_map_local, data_local)

    return best_solution
end

function BB(function_potential_local, rhythm_map_local, data_local)
        
    # Initialize global search variables
    best_solution = []
    best_score = 0
    total_depth = size(data_local, 2)
    
    # Value larger than or equal to the largest value in set
    upper_bound = 1000
    # Value smaller than or equal to the smallest value in set
    lower_bound = 0
    
    activeset = [function_potential_local[1]]
    current_best = 0
    current_solution = []
    println(activeset)
    
    
    #while !isempty(activeset)
        
        #active_node = pop!(activeset)
        #current_solution = [active_node]
        # Generate children of active_node and corresponding optimistic bounds
        
        
        
    #end
end

test = prepare(data[:, 1:16], functions, function_potential)

#search(test)

Any["S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O"]
Array{Any,1}[[(5, "T")]]


# # Brute Force Search Algorithm

In [156]:
function search(data, functions)
    
    Tonics = ["T", "T/3", "T/5", "T-5", "T64", "T54"]
    Subdominants = ["S", "S/3", "S/5", "S65", "S65/6", "S6"]
    Dominants = ["D", "D/3", "D/5", "D-5", "D64", "D5", "D54", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7"]
    Mediants = ["Ts", "Tm", "Ss", "Dm"]
    
    start = ["T", "T/3", "T/5"]
    leading_end = ["D", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7"]
    final = ["T"]
    
    # TODO: Rhythm analysis
    rhythm = (4, 1)    # Time signature, subdivisions per bar : (4, 1) = 4/4, 4 beats per bar, no subdivision of beats
    rhythm_map = rhythmify(rhythm)
    
    
    
    
    data_local = deepcopy(data[:, 1:20])
    functions_local = deepcopy(functions)
    
    best_solution = []
    best_score = 0
    function_potential_local = deepcopy(function_potential[1:size(data_local, 2)])
    
    ## Setup [T, ..., D, T] Frame
    filter!(x -> x[2] in start, function_potential_local[1])
    filter!(x -> x[2] in final, function_potential_local[end])
    filter!(x -> x[2] in leading_end, function_potential_local[end-1])
    
    ## Filter function potential by rhytmical rules
    
    
    
    ## Construct viable combinations from top ~5 
    current_solution = []
    
    for timestep in function_potential_local
        push!(current_solution, timestep[rand(1:5)])
    end
    
    
    
    ## Find an optimal starting pair of voicings
    
    best_cost = 1000
    local_best_previous = nothing
    local_best_current = nothing
    
    for first_voicing_list in function_permutations[current_solution[1][2]]
        previous_timestep = [current_solution[1], first_voicing_list]
        for second_voicing_list in function_permutations[current_solution[2][2]]
            current_timestep = [current_solution[2], second_voicing_list]
            local_cost = voicing_cost(previous_timestep, current_timestep)
            
            if local_cost < best_cost
                best_cost = local_cost
                local_best_previous = (previous_timestep, best_cost)
                local_best_current = (current_timestep, best_cost)
            end
        end
    end
    
    best_solution = [local_best_previous, local_best_current]
    
    for (i, timestep) in enumerate(current_solution[3:end])
        best_cost = 1000
        previous_timestep = best_solution[i+1][1]
        
        for voicing_list in function_permutations[current_solution[i+2][2]]
            current_timestep = [current_solution[i+2], voicing_list]
            local_cost = voicing_cost(previous_timestep, current_timestep)
            
            if local_cost < best_cost
                best_cost = local_cost
                push!(best_solution, (current_timestep, best_cost))
            end
        end
    end
        
    
    
    display(best_solution)
    
    return best_solution
    
end

test = search(data, functions)



MethodError: MethodError: no method matching rhythmify(::Tuple{Int64,Int64})
Closest candidates are:
  rhythmify(::Any, !Matched::Any) at In[148]:3

In [ ]:
test_notes = []
for voicing in test
    println(voicing[1][2])
    push!(test_notes, voicing[1][2])
end
println(test_notes)

In [99]:
## Convert back to piano roll

function makePianoRoll(notes)
    println(size(notes, 1))
    pianoroll = zeros(Int8, 128, size(notes,1))
    counter = 1
    for set in notes[:]
        for voice in set
            #println(voice)
            pianoroll[voice[1], counter] = 1
        end
        counter = counter + 1
        #println(pianoroll[:,1])
    end
    #println(findall(x->x==1, pianoroll))
    empty = zeros(Int8, 1, size(notes, 1))
    new = vcat(pianoroll, empty)
    display(new)
    writedlm("output\\test.csv",  new, ',')
end

makePianoRoll(test_notes)

129×33 Array{Int8,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0    

33


# # Construct a naive solution

# # NODAL

In [13]:
import Optim
SimulatedAnnealing(; neighbor = default_neighbor!,
                    T = default_temperature,
                    p = kirkpatrick)

┌ Info: Recompiling stale cache file C:\Users\nsthtz\.julia\compiled\v1.1\Optim\R5uoh.ji for Optim [429524aa-4258-5aef-a3af-852621145aeb]
└ @ Base loading.jl:1184


UndefVarError: UndefVarError: default_neighbor! not defined